In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn import preprocessing
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import tensorflow as tf

In [2]:
df = pd.read_csv('Processed_Data.csv', engine = 'python')
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140253,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.5,Positive,congratulation winner participant olympics win...
140254,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.5,Negative,hope wrong see tweet government
140255,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.0,Neutral,tokyo pass baton paris strangest ever olympic ...
140256,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.0,Neutral,paris plan deliver inclusive youth centre gend...


In [3]:
df = df.dropna()
df.shape

(139707, 6)

In [6]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(df['class'].values)
np.unique(labels)

array([0, 1, 2])

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(df.processed_tweets.values, labels, 
 random_state=42, 
 test_size=0.2)

In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-10-01 11:39:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-10-01 11:39:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-10-01 11:39:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [12]:
embeddings_index = {}
f = open('/content/glove.6B.100d.txt',encoding='utf8')
for line in tqdm(f):
 values = line.split()
 word = values[0]
 coefs = np.asarray(values[1:], dtype='float32')
 embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

400000it [00:11, 33392.65it/s]

Found 400000 word vectors.


In [13]:
VOCABULARY_SIZE = 2000
MAX_LENGTH = 60

In [14]:
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer.fit_on_texts(list(xtrain) + list(xtest))

In [17]:
xtrain_sequence = tokenizer.texts_to_sequences(xtrain)
xtest_sequence = tokenizer.texts_to_sequences(xtest)

In [18]:
xtrain_padding = sequence.pad_sequences(xtrain_sequence, maxlen=MAX_LENGTH)
xtest_padding = sequence.pad_sequences(xtest_sequence, maxlen=MAX_LENGTH)
word_index = tokenizer.word_index

In [21]:
embedding_matrix = np.zeros((len(word_index) + 1,100))
for word, i in tqdm(word_index.items()):
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

100%|██████████| 42086/42086 [00:00<00:00, 264676.08it/s]


In [27]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
 100,
 weights=[embedding_matrix],
 input_length=MAX_LENGTH,
 trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 100)           4208700   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 60, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              103424    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)             

In [29]:
ytrain_encode = np_utils.to_categorical(ytrain)
ytest_encode = np_utils.to_categorical(ytest)

In [32]:
history = model.fit(xtrain_padding, y=ytrain_encode, batch_size=512, epochs=10, verbose=1, validation_data=(xtest_padding, ytest_encode))

Epoch 1/10
219/219 [==============================] - 217s 991ms/step - loss: 0.5006 - accuracy: 0.8071 - val_loss: 0.4233 - val_accuracy: 0.8508
Epoch 2/10
219/219 [==============================] - 217s 990ms/step - loss: 0.4937 - accuracy: 0.8118 - val_loss: 0.4178 - val_accuracy: 0.8531
Epoch 3/10
219/219 [==============================] - 218s 994ms/step - loss: 0.4908 - accuracy: 0.8127 - val_loss: 0.4144 - val_accuracy: 0.8527
Epoch 4/10
219/219 [==============================] - 217s 990ms/step - loss: 0.4873 - accuracy: 0.8144 - val_loss: 0.4140 - val_accuracy: 0.8549
Epoch 5/10
219/219 [==============================] - 217s 993ms/step - loss: 0.4854 - accuracy: 0.8156 - val_loss: 0.4183 - val_accuracy: 0.8536
Epoch 6/10
219/219 [==============================] - 217s 992ms/step - loss: 0.4819 - accuracy: 0.8172 - val_loss: 0.4078 - val_accuracy: 0.8570
Epoch 7/10
219/219 [==============================] - 217s 990ms/step - loss: 0.4785 - accuracy: 0.8177 - val_loss: 0.4069 -

In [33]:
model.evaluate(xtest_padding, ytest_encode)

874/874 [==============================] - 22s 25ms/step - loss: 0.3981 - accuracy: 0.8619


[0.3981074392795563, 0.8618566989898682]